# RSOME for Robust Optimization

(rsome_ro:ro_framework)=
## General Framework for Robust Optimization Models

The `rsome.ro` module within RSOME is specifically crafted for robust optimization problems. This module features specialized modeling tools that enable the specification of random variables, uncertainty sets, and objective functions or constraints, accounting for worst-case realizations stemming from the uncertainty set. Let $\pmb{z}\in\mathbb{R}^J$ represent a vector of random variables, $\pmb{x}\in\mathbb{R}^{I_x}$ denote the here-and-now decision made before the uncertainty unfolds, and $\pmb{y}(\pmb{z})\in\mathbb{R}^{I_y}$ signify the wait-and-see decision made afterwards. Models supported by the `ro` module can be expressed in the following general format:

$$
\begin{align}
\min~&\max\limits_{\pmb{z}\in\mathcal{Z}_0}\left\{
\pmb{a}_0^{\top}(\pmb{z})\pmb{x} + \pmb{b}_0^{\top}\pmb{y}(\pmb{z}) + \pmb{c}_0(\pmb{z})
\right\} \\
\text{s.t.}~&\max\limits_{\pmb{z}\in\mathcal{Z}_m}\left\{
\pmb{a}_m^{\top}(\pmb{z})\pmb{x} + \pmb{b}_m^{\top}\pmb{y}(\pmb{z}) + \pmb{c}_m(\pmb{z})
\right\} \leq 0 &&\forall m \in \mathcal{M}\\
&y_i \in \mathcal{L}(\mathcal{J}_i) &&\forall i \in [I_y] \\
&\pmb{x}\in\mathcal{X}.
\end{align}
$$

In this context, $\mathcal{X}\subseteq \mathbb{R}^{I_x}$ represents a conic-representable feasible set for $\pmb{x}$, $\pmb{b}_m \in \mathbb{R}^{I_y}$ (for $m\in\mathcal{M}_1 \cup \{0\}$) serves as fixed parameters for $\pmb{y}$, and the uncertain parameters $\pmb{a}_m(\pmb{z})$ and $c_m(\pmb{z})$ are characterized as affine mappings of the random variable $\pmb{z}$.

$$
\pmb{a}_m(\pmb{z}) := \pmb{a}_m^0 + \sum\limits_{j\in[J]}\pmb{a}_m^jz_j \text{   and   }c_m(\pmb{z}) := c_m^0 + \sum\limits_{j\in[J]}c_m^jz_j,
$$

where $\pmb{a}_m^j\in\mathbb{R}^{I_x}$ and $c_m^j\in\mathbb{R}$, indexed by $j\in[J]\cup\{0\}$ and $m\in\mathcal{M}\cup \{0\}$, are proper coefficients. 

The wait-and-see decision $\pmb{y}$ potentially representing an arbitrary functional dependence on the uncertainty realization $\pmb{z}$, is inherently infinite-dimensional, posing challenges for optimization. To enhance tractability, a prevalent technique in robust optimization is the adoption of a linear decision rule (or affine decision rule). This involves constraining $\pmb{y}$ to simpler and more easily optimized affine functions, expressed in the following form:

$$
\mathcal{L}(\mathcal{J}) := \left\{
y: \mathbb{R}^{|\mathcal{J}|} \mapsto \mathbb{R}\left|
y(\pmb{z}) = y^0 + \sum\limits_{j\in\mathcal{J}}y^jz_j
\right.
\right\}.
$$

The RSOME package offers comprehensive algebraic tools for the explicit definition of the random variable $\pmb{z}$, uncertainty sets $\mathcal{Z}_m$, adaptive decision $\pmb{y}$, and its corresponding affine adaptive rules $\mathcal{L}(\mathcal{J}_i)$. These tools enable users to specify the worst-case objective and constraints given in the robust optimization framework above. 

(rsome_ro:rvar_uset)=
## Random Variables and Uncertainty Sets

Similar to the array-like decision variables, random variables of RSOME robust models are also defined as arrays using the `rvar()` method. The shape of the variable array is determined by an input integer or tuple, as illustrated by the examples below.

In [1]:
from rsome import ro

model = ro.Model()

u = model.rvar(3)                              # 3 random variables as a 1D array
v = model.rvar((3, 5))                         # 3x5 random variables as a 2D array
w = model.rvar((2, 3, 4, 5))                   # 2x3x4x5 random variables as a 4D array

All array operations, convex functions, and NumPy-style syntax for decision variables can also be applied to random variables in defining uncertainty sets. For example, let $\pmb{z}\in\mathbb{R}^5$ be the vector of five random variables, an uncertainty set $\mathcal{Z}_0=\left\{\pmb{z}\left|\|\pmb{z}\|_{\infty}\leq 1, \|\pmb{z}\|_1 \leq 1.5 \right.\right\}$ can be defined as `z_set0` in the following code segment.

In [2]:
from rsome import ro
from rsome import norm
import numpy as np

model = ro.Model()                              # create a model object

z = model.rvar(5)                               # 5 random variables as an 1D array

z_set0 = (norm(z, np.inf) <= 1,                 # the infinity-norm constraint
          norm(z, 1) <= 1.5)                    # the one-norm constraint

Note that an uncertainty set is a collection of constraints, written as an iterable Python object, such as tuple or list. These constraints are then used in specifying the worst-case objective function and constraints, which are introduced in the next section.

(rsome_ro:ro_worst_case)=
## The Worst-Case Objective and Constraints

In addition to specifying a deterministic objective function using the `min()` or `max()` method, the RSOME model also accommodates the definition of the worst-case objective over a designated uncertainty set through the `minmax()` or `maxmin()` method. Taking the `minmax()` function as an example, it minimizes the worst-case objective function provided as the first input argument, over an uncertainty set $\mathcal{Z}_0$ defined by the remaining arguments, as demonstrated by the following code.

In [3]:
from rsome import ro
import rsome as rso

model = ro.Model()   

x = model.dvar(5)                                # 5 decision variables as a 1D array
z = model.rvar(5)                                # 5 random variables as a 1D array

zset0 = (rso.norm(z, np.inf) <= 1,               # first constraint of the uncertainty set
        rso.norm(z, 1) <= 1.5)                   # second constraint of the uncertainty set

model.minmax(x @ z,                              # minimize the objective 
             zset0)                              # over the uncertaitny set z_set0     

Alternatively, the uncertainty set can be defined by separate constraints given as the inputs of the `minmax()` method.

In [4]:
model = ro.Model()   

x = model.dvar(5)                                # 5 decision variables as a 1D array
z = model.rvar(5)                                # 5 random variables as a 1D array

model.minmax(x @ z,                              # minimize the objective 
             rso.norm(z, np.inf) <= 1,           # frist constraint of the uncertainty set
             rso.norm(z, 1) <= 1.5)              # second constraint of the uncertainty set 

The `maxmin()` function works in the similar way in maximizing the worst-case objective function over a given uncertainty set $\mathcal{Z}_0$. 

Similar to deterministic constraints, the worst-case constraints can be defined using the NumPy-style array operations, and the designated uncertainty set $\mathcal{Z}_0$ is specified through the `forall()` method of the constraint, as illustrated by the sample code below.

In [5]:
from rsome import ro
import rsome as rso

model = ro.Model()   

x = model.dvar(5)
z = model.rvar(5)

zset0 = norm(z) <= 1.5

model.max(x.sum())                               # maximize the objective function

model.st((x * z <= 2).forall(zset0))             # the worst-case constraint is satisfied over zset0
model.st((x*z + x >= 0).forall(zset0))           # the worst-case constraint is satisfied over zset0    

for i in range(1, 6):
    zsetm = norm(z, 2) <= i*0.5
    model.st((x <= z[:i].sum()).forall(zsetm))   # the worst-case constraint is satisfied over zsetm

Note that if the uncertainty set of a robust constraint is unspecified, then by default, its uncertainty set is $\mathcal{Z}_0$, defined by the `minmax()` or `maxmin()` methods for the worst-case objective. The sample code above is hence equivalent to the following code segment.

In [6]:
from rsome import ro
import rsome as rso

model = ro.Model()   

x = model.dvar(5)
z = model.rvar(5)

zset0 = norm(z) <= 1.5

model.maxmin(x.sum(),                            # maximize the objective function
             zset0)                              # the default uncertainty set is defined to be zset0

model.st(x * z <= 2)                             # the worst-case constraint is satisfied over zset0
model.st(x*z + x >= 0)                           # the worst-case constraint is satisfied over zset0    

for i in range(1, 6):
    zsetm = norm(z, 2) <= i*0.5
    model.st((x <= z[:i].sum()).forall(zsetm))   # the worst-case constraint is satisfied over zsetm

The provided code demonstrates that the Python version of RSOME excels in specifying various uncertainty sets $\mathcal{Z}_m$, where $m\in\mathcal{M}\cup {0}$ denotes the objective function (for $m=0$) and each constraint (for $m \in \mathcal{M}$). This framework exhibits greater flexibility compared to its MATLAB counterpart as introduced by {cite:ps}`Chen_Sim_Xiong_2020robust`. It is adept at addressing a great variety of robust models, including the distributional interpretation of robust formulation in {cite:ps}`Xu_Caramanis_Mannor2012distributional`, the concept of Pareto robustly optimal solutions as discussed in {cite:ps}`De_Brekelmans_Den2016impact`, and the sample robust optimization models proposed by {cite:ps}`Bertsimas_Shtern_Surt2022two`.

(rsome_ro:ro_decision_rule)=
## Linear Decision Rules for Adaptive Decision-Making

In this section, we delve into addressing the wait-and-see decision $\pmb{y}$ that appears in multi-stage robust models. These wait-and-see decisions are established as arrays with specified shapes by invoking the `ldr()` method of the RSOME `ro` model object. This approach closely mirrors the methods previously introduced for creating arrays of decision and random variables, as illustrated by the sample code below.

In [7]:
from rsome import ro

model = ro.Model()

x = model.ldr((2, 4))                            # 2x4 decision rule variables as a 2D array
y = model.ldr((3, 5, 4))                         # 3x5x4 decision rule variables as a 3D array

print(x)
print(y)

2x4 decision rule variables
3x5x4 decision rule variables


Each adaptive decision $y_i$ has the capacity to affinely adapt to random variables through the specified linear decision rule $\mathcal{L}(\mathcal{J}_i)$, thereby enabling the derivation of an equivalent optimization format that is amenable to tractable solutions. The following code shows how the decision rules are specified using the indexing or slicing of variable arrays. 

In [8]:
u = model.rvar((2, 4))                           # 2x4 random variables as a 2D array
v = model.rvar(5)                                # 5 random variables as a 1D array

x.adapt(u)                                       # all elements of x depends on all u elements
y[2, 3:, :].adapt(u[0, 1])                       # y[2, 3:, :] depends on u[0, 1]
y[1, 3:, :].adapt(v[3:])                         # y[1, 3:, :] depends on v[3:]

Upon the creation of decision rules and the specification of their affine dependencies on random variables, the array operations and syntax introduced earlier can be employed for constructing the worst-case objective or constraints involving adaptive decisions. It is imperative to note that the affine dependency must be explicitly specified before utilizing decision rule variables in constraints; failure to do so will result in an error message.

Finally, after the model is solved, coefficients of a decision rule `y` that affinely adapts to random variable `z` could be accessed by the `get()` method. More specifically:

- `y.get()` returns the constant coefficients of the decision rule y. The returned array has the same shape as the decision rule array y.
- `y.get(z)` returns the linear coefficients of `y` with respect to the random variable `z`. The shape of the returned array is `y.shape + z.shape`, *i.e.*, the combination of dimensions of `y` and `z`. For example, if `c = y.get(z)` where `y.dim=2` and `z.dim=2`, the returned coefficients are presented as a four-dimensional array `c` and `c[i, j]` gives the linear coefficients of `y[i, j]` with respect to the random variable `z`.

(rsome_ro:sol_analysis)=
## Solution Analysis

After an RSOME model is successfully solved, affine or convex (concave) expressions of decision variables can be evaluated under the optimal solution as callable functions. Consider the adaptive robust optimization model below.

In [9]:
from rsome import ro
import rsome as rso
import numpy as np

c = np.array([0.1, 0.2, -0.3])
n = len(c)

model = ro.Model()
x = model.dvar(n)                                # x is a here-and-now decision

z = model.rvar()                                 # z is a random variable
uset = (z >= 0, z <= 1)                          # uncertainty set

y = model.ldr()                                  # y is an affine decision rule
y.adapt(z)                                       # y affinely adapts to z

obj = c@x + 2*y                                  # objective function
model.maxmin(obj, uset)                          # define the objective function
model.st(rso.norm(x, 1) <= 1)                    # define a constraint
model.st(y == z + 2.5)                           # define another constraint

model.solve()                                    # solve the model using the default solver

Being solved by the default LP solver...
Solution status: 0
Running time: 0.0024s


Apparently, the optimal solution of x is an array $(0, 0, -1)$, and in the following code segment, expressions `lin_expr`, `abs_expr`, and `norm_expr` are callable functions and their values are evaluated under the optimal value of `x`.

In [10]:
lin_expr = 5.0 - x.sum()                         # a linear expression of x 
abs_expr = 5.0 - abs(x.sum())                    # an expression of absolute values
norm_expr = rso.norm(x, 1)                       # a one-norm expression

print(f'{lin_expr()}, {abs_expr()}, {norm_expr()}')

6.0, 4.0, 1.0


As a special case, `x()` is equivalent to `x.get()`, which returns the optimal value of x.

Affine expressions of decision rules or expressions involving random variables can also be evaluated as similar callable functions. The only difference is that values of such expressions are also affected by the realized values of random variables, so we may need to specify the realizations of random variables in calling these functions. For example, the optimal decision rule y in the model above can be written as $y=z + 2.5$, its value under $z=0.5$ is evaluated using the following code,

In [11]:
print(y(z.assign(0.5)))

3.0


where the realization of a random variable is specified by the `assign()` method. The similar approach can be applied to the variable `obj` defined as the objective function of the model, as shown below.

In [12]:
print(obj(z.assign(1.5)))                        # The value of obj = c@x + 2*y when z = 1.5

8.3


If the realization of a random variable is unspecified, its value is assumed to be zero by default in evaluating the expressions. As a result, `y()` is equivalent to `y.get()` as they both return the constant term of the optimal linear decision rule.

(rsome_ro:examples)=
## Application Examples
- Robust Portfolio
- Conditional Value-at-Risk in Robust Portfolio Management
- Adaptive Robust Lot-Sizing
- Joint Production-Inventory Management
- Robust Vehicle Pre-Allocation

<br>
<br>

<font size="5">Reference</font>

```{bibliography}
:filter: docname in docnames
```